In [17]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import joblib
import time
import ta
import talib

In [18]:
# -------------------------
# CONFIG
# -------------------------
API_KEY = ""
API_SECRET = ""

symbol = "BTC/USDT"
model_timeframe = "15m"   # ML model trained on 15m candles
price_timeframe = "1s"    # execution monitored on 1-second candles
limit = 200
window_size = 10

FEATURES = ['RSI', 'EMA12', 'EMA26', 'MACD', 'Signal', 'Histogram', 'DEMA9', 'SMA', 'TSI', '%K', '%D']

SAVED_MODEL = "greg_tech_5.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.001       # TP % per grid leg
atr_multiplier = 1.5     # ATR multiplier for SL
position_fraction = 1.0
check_interval_seconds = 1
simulate_only = True

atr_period = 14

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
# -------------------------
# MODEL
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

model = CryptoLSTM(input_dim=len(FEATURES), hidden_dim=128, num_layers=4, output_dim=3)
model.load_state_dict(torch.load(SAVED_MODEL))
model.to(device)
model.eval()
scaler = joblib.load(SCALER_FILE)

In [21]:
import ccxt

In [22]:
# -------------------------
# EXCHANGE
# -------------------------
exchange = ccxt.binance({'enableRateLimit': True})

In [23]:
def compute_tsi(close, r1=25, r2=13):
    delta = close.diff()
    ema1 = delta.ewm(span=r1, adjust=False).mean()
    ema2 = ema1.ewm(span=r2, adjust=False).mean()

    abs_delta = delta.abs()
    abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
    abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()

    tsi = 100 * (ema2 / abs_ema2)
    return tsi


In [24]:
import ta
def compute_indicators(df):
    out = {}
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    sma_window = 3
    out['SMA'] = ta.trend.sma_indicator(df['close'], window=sma_window)
    out['TSI'] = compute_tsi(df['close'])
    period = 14
    smooth_k = 3
    smooth_d = 3

    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()

    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    
    feat_df = pd.DataFrame(out, index=df.index)
    return feat_df


In [25]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df


In [26]:
# -------------------------
# PREDICTION
# -------------------------
def predict_label_from_window(window_features):
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        out = model(x)
        probs = torch.softmax(out, dim=1).cpu().numpy()[0]
        # Reduce HOLD probability
        probs[1] *= 0.2
        probs = probs / probs.sum()
        pred = np.argmax(probs)
    return pred  # 0=down, 1=hold, 2=up


In [27]:
# -------------------------
# TRADING STATE
# -------------------------
balance = 1000.0
position = 0
entry_price = None
trade_size = 0.0
current_tp = None
current_sl = None

In [28]:
# -------------------------
# ATR
# -------------------------
def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 1)
    df['prev_close'] = df['close'].shift(1)
    df['TR'] = df.apply(lambda row: max(
        row['high'] - row['low'],
        abs(row['high'] - row['prev_close']),
        abs(row['low'] - row['prev_close'])
    ), axis=1)
    df['ATR'] = df['TR'].rolling(atr_period).mean()
    return df['ATR'].iloc[-1]

def update_tp_sl(price, trend='up'):
    global entry_price, current_sl, current_tp, trade_size
    atr_now = get_current_atr()
    trade_size = balance * position_fraction
    entry_price = price
    if trend == 'up':
        current_sl = price - atr_now * atr_multiplier
        current_tp = price * (1 + min_tp_pct)
    elif trend == 'down':
        current_sl = price + atr_now * atr_multiplier
        current_tp = price * (1 - min_tp_pct)
    else:
        # hold mode, soft grid
        current_sl = price - atr_now * 0.5
        current_tp = price + atr_now * 0.5

In [29]:
# -------------------------
# EXECUTE ORDERS
# -------------------------
def execute_buy(price):
    global position
    position = 1
    update_tp_sl(price, trend='up')
    print(f"[BUY] price={price:.2f}, SL={current_sl:.2f}, TP={current_tp:.2f}")

def execute_sell_close(price):
    global position, balance, trade_size
    if position == 1:
        pnl = trade_size * ((price - entry_price) / entry_price)
        balance += pnl
        print(f"[SELL] price={price:.2f}, P&L={pnl:.2f}, balance={balance:.2f}")
    position = 0

In [30]:
# -------------------------
# MICRO GRID
# -------------------------
def run_micro_grid(price, trend='up'):
    # Simulate 3 small buy/sell levels above/below current price
    grid_levels = []
    atr_now = get_current_atr()
    if trend == 'up':
        for i in range(1,4):
            grid_levels.append(price + i * atr_now*0.3)
    elif trend == 'down':
        for i in range(1,4):
            grid_levels.append(price - i * atr_now*0.3)
    else:
        # hold soft grid
        for i in [-1,0,1]:
            grid_levels.append(price + i * atr_now*0.2)
    print(f"[GRID-{trend.upper()}] Levels: {[round(l,2) for l in grid_levels]}")


In [31]:
# -------------------------
# MAIN LOOP
# -------------------------
back_df = fetch_latest_ohlcv(symbol, model_timeframe, 30 + window_size)
back_feat = compute_indicators(back_df).dropna()
preds = [predict_label_from_window(back_feat[FEATURES].values[i-window_size:i]) for i in range(window_size, len(back_feat))]
last_reversal = next((p for p in reversed(preds) if p != 1), None)

if last_reversal == 2:
    print("Last model prediction was UPTREND -> ENTER INITIAL POSITION")
    last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]
    execute_buy(last_price)
    run_micro_grid(last_price, trend='up')

elif last_reversal == 0:
    print("Last model prediction was DOWNTREND -> ENTER SHORT GRID (fake reversal long)")
    last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]
    position = 1
    entry_price = last_price
    update_tp_sl(last_price, trend='down')
    run_micro_grid(last_price, trend='down')

else:
    print("Last model prediction is HOLD -> soft grid")
    last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]
    run_micro_grid(last_price, trend='hold')

print("Starting hybrid ML + ATR grid logic. Press Ctrl+C to stop.")

try:
    while True:
        df_ohlcv = fetch_latest_ohlcv(symbol, model_timeframe, limit)
        feat_df = compute_indicators(df_ohlcv).dropna()
        if len(feat_df) < window_size:
            print("Not enough data. Waiting...")
            time.sleep(10)
            continue

        last_rows = feat_df[FEATURES].values[-window_size:]
        pred = predict_label_from_window(last_rows)
        last_price = fetch_latest_ohlcv(symbol, price_timeframe, 2)['close'].values[-1]

        trend = 'hold'
        if pred == 2:
            trend = 'up'
        elif pred == 0:
            trend = 'down'

        print(f"[{pd.Timestamp.utcnow()}] Predicted: {pred}, Price: {last_price:.2f}, Balance: {balance:.2f}, Position: {position}")

        # Enter position if trend and no current position
        if trend == 'up' and position == 0:
            execute_buy(last_price)
        elif trend == 'down' and position == 0:
            # short-selling could be implemented here
            position = 1
            entry_price = last_price
            update_tp_sl(last_price, trend='down')

        # Stop loss & take profit
        if position == 1:
            if trend == 'up' and last_price <= current_sl:
                print("STOP LOSS HIT")
                execute_sell_close(last_price)
            elif trend == 'down' and last_price >= current_sl:
                print("STOP LOSS HIT (short)")
                execute_sell_close(last_price)
            elif (trend == 'up' and last_price >= current_tp) or (trend == 'down' and last_price <= current_tp):
                pnl = trade_size * ((current_tp - entry_price) / entry_price)
                balance += pnl
                print(f"TAKE PROFIT HIT: P&L={pnl:.2f}, Balance={balance:.2f}")
                update_tp_sl(current_tp, trend=trend)

        # Run micro-grid per trend
        run_micro_grid(last_price, trend=trend)
        time.sleep(check_interval_seconds)

except KeyboardInterrupt:
    print("Stopped by user. Final balance:", balance)

Last model prediction was UPTREND -> ENTER INITIAL POSITION
[BUY] price=119708.92, SL=119479.74, TP=119828.63
[GRID-UP] Levels: [np.float64(119754.76), np.float64(119800.59), np.float64(119846.43)]
Starting hybrid ML + ATR grid logic. Press Ctrl+C to stop.
[2025-10-03 07:37:10.147192+00:00] Predicted: 1, Price: 119708.92, Balance: 1000.00, Position: 1
[GRID-HOLD] Levels: [np.float64(119678.36), np.float64(119708.92), np.float64(119739.48)]
[2025-10-03 07:37:12.548116+00:00] Predicted: 1, Price: 119708.91, Balance: 1000.00, Position: 1
[GRID-HOLD] Levels: [np.float64(119678.35), np.float64(119708.91), np.float64(119739.47)]
[2025-10-03 07:37:14.954083+00:00] Predicted: 1, Price: 119708.92, Balance: 1000.00, Position: 1
[GRID-HOLD] Levels: [np.float64(119678.36), np.float64(119708.92), np.float64(119739.48)]
[2025-10-03 07:37:17.345349+00:00] Predicted: 1, Price: 119708.91, Balance: 1000.00, Position: 1
[GRID-HOLD] Levels: [np.float64(119678.35), np.float64(119708.91), np.float64(119739.

RequestTimeout: binance GET https://api.binance.com/api/v3/klines?interval=15m&limit=200&symbol=BTCUSDT